In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import re
import seaborn as sns
import word2number
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from nltk.corpus import wordnet as wn
from word2number import w2n
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords #<- For calling the know stopwords in english (e.g, articles, connectors)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Vass\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\Vass\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\Vass\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning,

In [4]:
df = pd.read_csv("synonyms.csv")

df.head()

,lemma,part_of_speech,synonyms
0,.22-caliber,adjective,.22 caliber;.22 calibre;.22-calibre
1,.22-calibre,adjective,.22 caliber;.22-caliber;.22 calibre
2,.22 caliber,adjective,.22-caliber;.22 calibre;.22-calibre
3,.22 calibre,adjective,.22 caliber;.22-caliber;.22-calibre
4,.38-caliber,adjective,.38 caliber;.38 calibre;.38-calibre


In [32]:
def numbers(x):
    try:
        return w2n.word_to_num(x)
    except (ValueError, TypeError, IndexError):
        return np.nan

In [42]:
df["number"] = df["lemma"].apply(lambda x: numbers(x))

In [96]:
df["lemma"].unique().shape

(117204,)

In [97]:
th_rows = [f"{i}th" for i in range(1000)]
rd_rows = [f"{i}rd" for i in range(1000)]
nd_rows = [f"{i}nd" for i in range(1000)]
st_rows = [f"{i}st" for i in range(1000)] 
number_rows = [f"number {i}" for i in range(1000)]
years_rows = [f"{i}s" for i in range(0,2021,10)]
days_rows = [f"{i+1} days" for i in range(366)]
final_rows = ["24-hour","-karat"]

final_rows.extend(th_rows)
final_rows.extend(number_rows)
final_rows.extend(rd_rows)
final_rows.extend(nd_rows)
final_rows.extend(st_rows)

In [99]:
no_numbers = df[(df["lemma"].str.isnumeric() == False)&(df["number"].isna() == True)&(df["lemma"].str.contains('|'.join(final_rows)) == False)]

In [101]:
no_numbers["lemma"].unique().shape

(116285,)

In [102]:
ps = PorterStemmer()
no_numbers["stemmed"] = no_numbers["lemma"].apply(lambda w: ps.stem(w) if isinstance(w, str) else None)

C:\Users\Vass\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [103]:
no_numbers[no_numbers.duplicated(subset=["stemmed","part_of_speech"]) == True].shape

(10754, 5)

In [104]:
no_numbers[no_numbers.duplicated(subset=["stemmed","part_of_speech"]) == True]["lemma"].unique().shape

(9199,)

In [105]:
remove_stemming = no_numbers[no_numbers.duplicated(subset=["stemmed","part_of_speech"]) == False]

In [106]:
remove_stemming["lemma"].unique().shape

(108495,)

In [107]:
remove_stemming["split"] = remove_stemming["synonyms"].str.replace("|",";").apply(lambda x: x.split(";") if isinstance(x, str) else None)

C:\Users\Vass\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Need to pair up words that are synonyms.

In [108]:
remove_stemming.shape

(115228, 6)

In [109]:
stops = stopwords.words('english')

In [110]:
# filter out obvious stop words such as "have" "and"
no_stops = remove_stemming[remove_stemming["lemma"].apply(lambda x: (x in list(STOPWORDS) or x in stops))==False]

In [111]:
no_stops["lemma"].unique().shape

(108411,)

In [112]:
with_syns = no_stops[no_stops["synonyms"].isna() ==False]

In [113]:
with_syns["lemma"].unique().shape

(108410,)

In [93]:
with_syns[with_syns.split.apply(lambda x: "bad" in x)]

,lemma,part_of_speech,synonyms,number,stemmed,split
2464,big,satellite,bad|large;prominent|heavy|boastful;braggart;br...,NaN,big,"[bad, large, prominent, heavy, boastful, bragg..."
2470,bigger,satellite,larger|big|bad;big|big|big;large;prominent|big...,NaN,bigger,"[larger, big, bad, big, big, big, large, promi..."
5370,defective,satellite,faulty|bad,NaN,defect,"[faulty, bad]"
7907,forged,satellite,bad,NaN,forg,[bad]
9202,high-risk,satellite,bad;risky;speculative,NaN,high-risk,"[bad, risky, speculative]"
15667,regretful,adjective,sorry;bad,NaN,regret,"[sorry, bad]"
16014,risky,satellite,hazardous;wild|bad;high-risk;speculative,NaN,riski,"[hazardous, wild, bad, high-risk, speculative]"
17645,sorry,adjective,regretful;bad,NaN,sorri,"[regretful, bad]"
17781,speculative,satellite,bad;risky;high-risk|notional|inquisitive;quest...,NaN,specul,"[bad, risky, high-risk, notional, inquisitive,..."
17875,spoiled,satellite,spoilt|bad;spoilt,NaN,spoil,"[spoilt, bad, spoilt]"


In [114]:
pairs = []

for lem in no_stops["lemma"]:
    
    if with_syns[with_syns.split.apply(lambda x: lem in x)].shape[0] > 0:
        
        for comp_lem in with_syns[with_syns.split.apply(lambda x: lem in x)]["lemma"]:
            if lem != comp_lem:
                comp_pair = [lem, comp_lem]
                pairs.append(comp_pair)

In [115]:
len(pairs)
# 367754 with numbers included
# 367024 without numbers v1
# 250440 after stemming
# 245308 with no numbers as words

245308

In [116]:
np_pairs = np.array(pairs)
edges_raw = pd.DataFrame(np_pairs)

In [117]:
edges_raw.shape

(245308, 2)

Now that all pairs are in a dataframe must get rid of duplicates.

In [119]:
df1 = pd.DataFrame(np.sort(edges_raw[[0,1]], axis=1))

edges_raw1 = edges_raw[~df1.duplicated()]

In [130]:
edges_raw1[0].unique().shape

(46872,)

In [131]:
edges_raw1.head()

,0,1
0,.22-caliber,.22-calibre
1,.22-caliber,.22 caliber
2,.22-caliber,.22 calibre
4,.22-calibre,.22 caliber
5,.22-calibre,.22 calibre


In [135]:
edges_raw1.rename(columns={0:"Source",1:"Target"},inplace=True)
edges_raw1.to_csv("edges.csv",index=False)

C:\Users\Vass\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [125]:
# Convert columns into list of tuples
tup_edges = list(zip(edges_raw1[0], edges_raw1[1]))

# list of nodes and associated connections
edges = [t for t in (set(tuple(i) for i in tup_edges))]

In [127]:
# create networkx graph
Gx = nx.Graph()
Gx.add_edges_from(edges)


In [129]:
print(len(Gx.nodes))

print(len(Gx.edges))

75253
116131


In [10]:
df["lemma"].apply(lambda x: x if x in stopwords)

SyntaxError: invalid syntax (<ipython-input-10-be1d9a5ab270>, line 1)

In [17]:
stops = stopwords.words('english')

In [24]:
hmmm = df[df["lemma"].apply(lambda x: (x in list(STOPWORDS) or x in stops))==True]

In [25]:
hmmm.sort_values(by="lemma")

,lemma,part_of_speech,synonyms
24708,a,noun,angstrom;angstrom unit;A|vitamin A;antiophthal...
328,about,satellite,astir
329,about,adverb,approximately;close to;just about;some;roughly...
330,above,adverb,supra|higher up;in a higher place;to a higher ...
740,after,adverb,subsequently;later;afterwards;afterward;later on
...,...,...,...
119208,will,verb,bequeath;leave
21848,won,noun,South Korean won|North Korean won
21849,won,verb,win|acquire;win;gain|gain;advance;win;pull ahe...
120177,www,noun,World Wide Web;WWW;web
